In [1]:
%cd ~/Documents/cvd-predictor
import polars as pl
from CVD.models.preprocesser import PreProcesser
csv_path = "data/raw/LLCP2023.csv"
parquet_path = "data/intermediate/LLCP2023.parquet"

C:\Users\Lorenzo\Documents\cvd-predictor


In [2]:
pl.read_csv(csv_path).write_parquet(parquet_path)
df: pl.DataFrame = pl.read_parquet(parquet_path)

In [3]:
variables: pl.DataFrame = pl.read_csv("data/raw/selected_variables.csv")
df: pl.DataFrame = df[variables["Variable"]]
df.columns = variables["Renamed"]

In [4]:
preprocesser: PreProcesser = PreProcesser()
df: pl.DataFrame = preprocesser.merge_columns(df)
df = preprocesser.encode_columns(df)
df = preprocesser.merge_cvd(df)

In [5]:
df: pl.DataFrame = df.drop_nulls(["CVD"])
df.write_csv("data/intermediate/heart_cdc_2023.csv")
df

Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,SmokerStatus,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HaveHighCholesterol,Sensory Impairments,Vaccinated,Mobility,CVD
i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i64,i8,f64,f64,f64,i8,i8,i8,i8,i8,i8
0,2,null,null,2,2,2,2,2,2,2,2,1,4,13,1.6,78.02,30.47,2,0,0,0,1,0
0,2,null,null,2,1,2,2,2,1,2,1,3,4,13,1.45,59.87,28.56,2,1,1,1,0,0
0,4,6,2,1,1,1,2,2,2,2,1,3,3,13,1.63,58.97,22.31,2,1,1,1,1,0
0,2,2,null,3,1,2,1,2,1,2,1,3,4,12,1.68,77.11,27.44,2,0,0,1,1,0
0,4,null,null,1,1,2,2,2,1,2,1,1,4,12,1.73,77.11,25.85,1,0,0,1,1,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,3,12,30,1,1,2,2,2,2,2,2,3,4,10,1.93,108.86,29.21,1,1,0,0,0,0
0,2,null,null,1,2,2,2,2,2,2,2,3,4,3,1.65,68.04,24.96,2,0,0,0,0,0
0,2,10,null,1,1,2,2,2,2,2,2,3,4,7,1.68,96.62,34.38,1,1,0,1,0,0
